In [90]:
import sys
sys.path.append("..")
import lib
from parse import *
import numpy as np


Task Description

In [44]:
def parse_line(line : str) -> list:
    parsed = findall("{:d},{:d}", line)
    parsed = [list(r) for r in parsed]
    return parsed

def parse_file(filename : str) -> list:
    lines = lib.read_file(filename)
    lines = [parse_line(line) for line in lines]
    return lines

parse_file("test_input.txt")

[[[498, 4], [498, 6], [496, 6]], [[503, 4], [502, 4], [502, 9], [494, 9]]]

In [135]:
def get_slice_limits(lines):
    list_x = np.array([500])
    list_y = np.array([0])
    for line in lines:
        line = np.array(line)
        list_x = np.append(list_x, line[:,0].flatten())
        list_y = np.append(list_y, line[:,1].flatten())
    list_x = np.array(list_x, dtype = int)
    list_y = np.array(list_y, dtype = int)
    return (min(list_x), max(list_x)), (min(list_y), max(list_y))

def get_field(xlim : tuple, ylim : tuple):
    field = np.zeros((xlim[1] - xlim[0] + 1, ylim[1] - ylim[0] + 1), dtype = int)
    # 0: air / -2 : source of sand / -1 : rested sand / 1 : rock
    center = (500 - xlim[0], 0)
    field[center] = -2
    return field, np.array(center)

def map_numbers(i):
    if i == 0: # air
        return '.'
    elif i == -2: # source of sand
        return '+'
    elif i == -1: # rested sand
        return 'o'
    elif i == 1: # rock
        return '#'
    else:
        raise ValueError

def get_partB_limits(xlim, ylim):
    new_x_min = 0
    new_x_max = 1000
    new_y_min = ylim[0]
    new_y_max = ylim[1] + 2
    return (new_x_min, new_x_max), (new_y_min, new_y_max)

class Slice():
    def __init__(self, filename : str, partB : bool = False):
        parsed_input = parse_file(filename)
        self.xlim, self.ylim = get_slice_limits(parsed_input)
        if partB: # hacky hack
            self.xlim, self.ylim = get_partB_limits(self.xlim, self.ylim)
        self.field, self.center = get_field(self.xlim, self.ylim)
        print(f"Center initialized at {self.center}")
        self.width = self.xlim[1] - self.xlim[0] + 1
        self.height = self.ylim[1] - self.ylim[0] + 1
        print(f"Width = {self.width} / Height = {self.height} / Shape = {self.field.shape}")
        def parse_field(lines : list):
            for line in lines:
                for idx in range(len(line) - 1):
                    start = line[idx] # e.g. 498,4
                    end = line[idx+1] # e.g. 498,6
                    if start[0] > end[0] or start[1] > end[1]:
                        temp = start
                        start = end
                        end = temp
                    self.field[start[0] - self.xlim[0] : end[0] - self.xlim[0] + 1, start[1] - self.ylim[0] : end[1] - self.ylim[0] + 1] = 1
            return
        parse_field(parsed_input)
        if partB: # init last row to rocks
            self.field[:,-1] = 1
        if not partB: # not readable
            print(self.__repr__())
        self.IndexError = False
        return

    def __repr__(self):
        output = map(map_numbers, self.field.T.flatten())
        output_str = ''
        i = 0
        for element in output:
            i += 1
            output_str += element
            if i == self.width:
                output_str += "\n"
                i = 0
        output_str += '\n'
        return output_str

    def produce_sand(self, verbose = False):
        position = self.center
        def out_of_box(pos):
            width, height = self.field.shape
            if pos[0] >= width or pos[0] < 0: # horizontally outside
                return True
            elif pos[1] >= height or pos[1] < 0: # vertically outside
                return True
            else:
                return False

        def propagate(pos):
            try:
                if self.field[tuple(pos + np.array([0,1]))] == 0: # free
                    pos = pos + np.array([0,1])
                elif self.field[tuple(pos + np.array([-1,1]))] == 0: # free
                    pos = pos + np.array([-1,1])
                elif self.field[tuple(pos + np.array([1,1]))] == 0: # free
                    pos = pos + np.array([1,1])
                else: # nothing free
                    return False, pos # -> set to rest
                return True, pos # -> continue
            except IndexError:
                self.IndexError = True
                return False, pos
        alive = True
        while alive and not out_of_box(position):
            alive, position = propagate(position)

        if out_of_box(position) or self.IndexError or tuple(position) == tuple(self.center):
            if self.IndexError:
                print(f"  IndexError at {position}.") if verbose else None
            elif tuple(position) == tuple(self.center):
                print(f"  New sand overlaps at {self.center} with source.") if verbose else None
                self.field[tuple(position)] = -1
            else:
                print(f"  Out of box at {position}.") if verbose else None
            return False # make sure pour_sand stops
        else: # -> not alive
            self.field[tuple(position)] = -1
            return True # continue pouring sand
            
    def pour_sand(self, verbose = False):
        print("Starting to pour sand...")
        while self.produce_sand():
            continue
        print(f"Finished pouring sand...")
        print(self.__repr__()) if verbose else None
        return

    def get_number_of_resting_sand(self):
        return np.sum(self.field == -1)


Solution Part A

In [133]:
def compute_partA(filename : str, verbose : bool = False):
    slice = Slice(filename)
    slice.pour_sand(verbose)
    return slice.get_number_of_resting_sand()

def solve_partA():
    result = compute_partA("test_input.txt", True)
    true_solution = 24
    assert result == true_solution, f"Part A faulty on test file... output = {result}"
    print("Part A works for test file, moving on to whole input...")
    result = compute_partA("input.txt", False)
    print(f"Answer: {result}")
    return

solve_partA()

Center initialized at [6 0]
Width = 10 / Height = 10 / Shape = (10, 10)
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
........#.
#########.


Starting to pour sand...
Finished pouring sand...
......+...
..........
......o...
.....ooo..
....#ooo##
...o#ooo#.
..###ooo#.
....oooo#.
.o.ooooo#.
#########.


Part A works for test file, moving on to whole input...
Center initialized at [49  0]
Width = 59 / Height = 164 / Shape = (59, 164)
.................................................+.........
...........................................................
...........................................................
...........................................................
...........................................................
...........................................................
...........................................................
...........................................................
..........................................

Solution Part B

In [136]:
def compute_partB(filename : str, verbose : bool = False):
    slice = Slice(filename, True)
    slice.pour_sand(verbose)
    return slice.get_number_of_resting_sand()

def solve_partB():
    result = compute_partB("test_input.txt", False)
    true_solution = 93
    assert result == true_solution, f"Part B faulty on test file... output = {result}"
    print("Part B works for test file, moving on to whole input...")
    result = compute_partB("input.txt", False)
    print(f"Answer: {result}")
    return

solve_partB()

Center initialized at [500   0]
Width = 1001 / Height = 12 / Shape = (1001, 12)
Starting to pour sand...
Finished pouring sand...
Part B works for test file, moving on to whole input...
Center initialized at [500   0]
Width = 1001 / Height = 166 / Shape = (1001, 166)
Starting to pour sand...
Finished pouring sand...
Answer: 25248
